# Duckietown NCTU - Tutorial 6: Drive mybot in Gazebo

By Brian Chuang, Jack Lin, and Nick Wang

## Import Packages

In [10]:
import numpy as np
import scipy as sp
import cv2
import time

from matplotlib import pyplot as plt
%matplotlib inline
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels

## ROS Setup

In [11]:
import sys
# rospy
sys.path.insert(0, '/opt/ros/indigo/lib/python2.7/dist-packages')
# rospkg
sys.path.insert(0, '/usr/lib/python2.7/dist-packages/')

# duckietown_msgs
duckietown_root = '../../'  # this file should be run from {duckietown_root}/turorials/python (otherwise change this line)
sys.path.insert(0, duckietown_root + 'catkin_ws/devel/lib/python2.7/dist-packages')

import rospy
from duckietown_msgs.msg import  Twist2DStamped
from geometry_msgs.msg import Twist # for cmd_vel


### you need to run ./run_gazebo for mybot

### initial a rosnode to send cmd_vel, similar to car_cmd in Duckietown

In [12]:
rospy.init_node("cmd_vel_jupyter",anonymous=False)
pub_car_cmd = rospy.Publisher("/cmd_vel",Twist,queue_size=1)


### define function for publishing car command 

In [13]:
# v: linear x, omega: angular z
def cmd_vel_command(v, omega, duration):
# Send stop command
    cmd_vel_msg = Twist()
    cmd_vel_msg.linear.x = v 
    cmd_vel_msg.angular.z = omega 
    pub_car_cmd.publish(cmd_vel_msg)
    rospy.sleep(duration)
    #rospy.loginfo("Shutdown")
    cmd_vel_msg.linear.x = 0
    cmd_vel_msg.angular.z = 0 
    pub_car_cmd.publish(cmd_vel_msg)   

## EXERCISE: Forward (F), Turn Left (L), or Turn Right (R)

Send commands and calibrate your duckiebot

### Ex1: Forward 0.5 Tile Width

In [16]:
cmd_vel_command(0.5, 0, 0.75)

### EX2: Turn 45 or 90 Degrees

In [18]:
cmd_vel_command(0.2, 4, 1.005)

### EX3: Concat the commands

In [19]:
class switch(object):
    def __init__(self, value):
        self.value = value
        self.fall = False
 
    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration
     
    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args: # changed for v1.5, see below
            self.fall = True
            return True
        else:
            return False

In [20]:
def concat_commands(concat):
    for i in range(len(concat)):
        primitives = concat[i]
        for case in switch(primitives):
            if case('S'):
                cmd_vel_command(0.5, 0, 0.25)
                break
            if case('L'):
                cmd_vel_command(0.2, 4, 0.82)
                break
            if case('R'):
                cmd_vel_command(0.2, -4, 0.78)
                break
            if case('B'):
                cmd_vel_command(-0.4, 0, 0.5)
                break

### example: overtaking

In [21]:
overtaking = "LSRSSSSRSLSS"
#overtaking = "LSRSSS"
concat_commands(overtaking)

### example: parking 

In [22]:
parking = "BBLBBB"
concat_commands(parking)